In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image


# Persiapan Data

In [ ]:
import pathlib
dataset_url = "/content/drive/MyDrive/basedata/train"
data_dir = pathlib.Path(dataset_url)

In [ ]:
img_height,img_width=180,180
batch_size=32
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  label_mode='categorical',
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 1200 files belonging to 6 classes.
Using 960 files for training.


In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  label_mode='categorical',
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 1200 files belonging to 6 classes.
Using 240 files for validation.


In [ ]:
class_names = train_ds.class_names
print(class_names)

['bmx', 'motor matic', 'motosport', 'mtb', 'suv', 'van']


# Melatih Dataset


In [ ]:
# Mendapatkan informasi tentang kelas dari ImageNetLabels
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
with open(labels_path) as f:
    labels = f.read().splitlines()

# Menampilkan 1000 kelas
for i, label in enumerate(labels):
    if i < 1000:
        print(f"Class {i}: {label}")
    else:
        break

In [ ]:
resnet_model = Sequential()

pretrained_model= tf.keras.applications.ResNet101(
                   include_top=False,
                   input_shape=(180,180,3),
                   pooling='avg',
                   classes=6,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(6, activation='sigmoid'))

In [ ]:
resnet_model.summary()

In [ ]:
resnet_model.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
epochs=10
history = resnet_model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

# Evaluasi Model

In [ ]:
fig1 = plt.gcf()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.axis(ymin=0.4,ymax=1)
plt.grid()
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.grid()
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
# Save Model
resnet_model.save_weights('sequential.h5')

# Prediksi Kelas

In [ ]:
import cv2

image_path = 'mtb.jpeg'
image=cv2.imread(str(image_path))
image_resized= cv2.resize(image, (img_height,img_width))
image=np.expand_dims(image_resized,axis=0)
print(image.shape)
PIL.Image.open(str(image_path))


In [ ]:
pred=resnet_model.predict(image)
print(pred)

In [ ]:
output_class_id = np.argmax(pred)
output_class = class_names[output_class_id]
score = np.array(pred)
print("Gambar tersebut adalah id kelas =", output_class_id)
print("Gambar tersebut adalah kelas =", output_class)
print("Skor dari gambar tersebut adalah =", score.max())

**==========================================================**

# Prediksi Kelas Sebelum Tahap *Transfer Learning*

In [ ]:
import torch
from torchvision.models import resnet101, ResNet101_Weights
from pprint import pprint
from PIL import Image

weights = ResNet101_Weights.DEFAULT
preprocess = weights.transforms()

resnet = resnet101(weights=weights)
img = Image.open('gambar1.png')
img = img.convert('RGB')
pprint(f"ukuran gambar sebelum preprocess= {img.size}")

img = preprocess(img)
pprint(f"Bentuk gambar sesudah preprocess = {img.shape}")

batch = img.unsqueeze(0)
# Unsqueeze(0) karena hanya memproses satu gambar
pprint(f"Batch gambar = {batch.shape}")

resnet.eval()
prediction = resnet(batch).squeeze(0).softmax(0)
class_id = prediction.argmax()
score = prediction[class_id].item()
label = weights.meta["categories"][class_id]
pprint(f"class prediction = {class_id}, Score = {score}, label={label}")